<div align='center'>
    <img src='https://radar.gr/wp-content/uploads/2022/06/220621125542_fake-news.jpg'><br>
    Solving the competition with Kaggle
    <a href='https://www.kaggle.com/datasets/morfifinka/fake-real-news-ru'>https://www.kaggle.com/datasets/morfifinka/fake-real-news-ru</a>
</div>

# 1.1 Loading libraries and parsing data

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import nltk
from nltk.corpus import stopwords
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
import warnings
# Turning off the warnings
warnings.filterwarnings('ignore')

In [2]:
# Loading data
test_stances = pd.read_csv("test_stances_unlebeledb.csv", index_col='Unnamed: 0')
test_bodies = pd.read_csv("test_bodies.csv", index_col='Unnamed: 0')
train_bodies = pd.read_csv("train_bodies.csv", index_col='Unnamed: 0')
train_stances = pd.read_csv("train_stances.csv", index_col='Unnamed: 0')

In [3]:
# Combining two training sample csv files into one file
count=0
for i in tqdm(range(train_stances.shape[0])):
    for j in range(train_bodies.shape[0]):
        if train_bodies.loc[j,'Body ID']==train_stances.loc[i,'Body ID']:
            train_stances.loc[i,'articleBody'] = train_bodies.loc[j,'articleBody']
            train_stances.loc[i,'articleBody1'] = train_bodies.loc[j,'articleBody1']
            train_stances.loc[i,'articleBody2'] = train_bodies.loc[j,'articleBody2']
    train_stances.to_csv('data_combined.csv',index=False)

  0%|          | 0/4408 [00:00<?, ?it/s]

In [4]:
data = pd.read_csv('data_combined.csv') # Write the new data to the file
data # Let's look at the data

,Body ID,Headline,Stance,Headline1,Headline2,articleBody,articleBody1,articleBody2
0,1,"['российский', 'бюджет', 'марте', 'недополучил...",agree,российский бюджет в марте недополучил более 30...,российский бюджет март недополучать миллиард р...,Разница между ожидаемыми по итогам марта нефте...,разница ожидаемыми итогам марта нефтегазовыми ...,разница ожидать итог март нефтегазовый доход ф...
1,2,"['банк', 'россии', 'решил', 'снизить', 'ключев...",agree,банк россии решил снизить ключевую ставку с 20...,банк россия решать снижать ключевой ставка,Совет директоров Банка России решил снизить кл...,совет директоров банка россии решил снизить кл...,совет директор банк россия решать снижать ключ...
2,3,"['мыс', 'идокопас', 'нато', 'назвали', 'первую...",disagree,мыс идокопас в нато назвали первую цель для н...,мыс идокопас нато называть первый цель начало ...,В НАТО составили наступательный план действий ...,нато составили наступательный план действий за...,нато составлять наступательный план действие з...
3,4,"['украине', 'прошли', 'празднования', '300', '...",disagree,на украине прошли празднования 300 летия побед...,украина проходить празднование летие победа ка...,Президент Владимир Зеленский принял в Полтаве ...,президент владимир зеленский принял полтаве па...,президент владимир зеленский принимать полтава...
4,5,"['минобороны', 'рф', 'заявило', 'ударах', 'рак...",agree,минобороны рф заявило об ударах ракетами «кали...,минобороны заявлять удар ракета калибр запорож...,"Минобороны России объявило, что ракетами больш...",минобороны россии объявило ракетами большой да...,минобороны россия объявлять ракета большой дал...
...,...,...,...,...,...,...,...,...
4403,4404,"['чиновница', 'призвавшая', '«найти', 'покарат...",disagree,чиновница призвавшая «найти и покарать предате...,чиновница призывать находить покарать предател...,Заместитель министра просвещения Ирина Карепин...,заместитель министра просвещения ирина карепин...,заместитель министр просвещение ирина карепин ...
4404,4405,"['«включим', 'программу', 'киселёва', 'покажем...",disagree,«включим программу киселёва и покажем телевизо...,включать программа киселев показывать телевизо...,Глава штабов Навального Леонид Волков призвал ...,глава штабов навального леонид волков призвал ...,глава штаб навальный леонид волк призывать сво...
4405,4406,"['китайские', 'власти', 'могут', 'казнить', 'з...",disagree,китайские власти могут казнить задержанного по...,китайский власть мочь казнить задержанный подо...,Один из менеджеров стройки космодрома Восточны...,менеджеров стройки космодрома восточный взят с...,менеджер стройка космодром восточный взять стр...
4406,4407,"['луганской', 'области', 'переполнены', 'морги...",agree,в луганской области переполнены морги хранить...,луганский область переполнять морг хранить тел...,Морги Луганской области переполнены телами пог...,морги луганской области переполнены телами пог...,морг луганский область переполнять тело погибш...


In [5]:
# Combining two test sample csv files into a single file
for i in tqdm(range(test_stances.shape[0])):
    for j in range(test_bodies.shape[0]):
        if train_bodies.loc[j,'Body ID']==train_stances.loc[i,'Body ID']:
            test_stances.loc[i,'articleBody'] = test_bodies.loc[j,'articleBody']
            test_stances.loc[i,'articleBody1'] = test_bodies.loc[j,'articleBody1']
            test_stances.loc[i,'articleBody2'] = test_bodies.loc[j,'articleBody2']
    test_stances.to_csv('data_test_combined.csv',index=False)

  0%|          | 0/1101 [00:00<?, ?it/s]

In [6]:
data_test = pd.read_csv('data_test_combined.csv') # Let's write the new data to the file
data_test # Let's look at the data

,Body ID,label,Headline,Headline1,Headline2,articleBody,articleBody1,articleBody2
0,1,0,"['лукашенко', 'пригрозил', 'литовским', 'танка...",лукашенко пригрозил литовским танкам белорусск...,лукашенко пригрозить литовский танк белорусски...,"Президент Беларуси Александр Лукашенко заявил,...",президент беларуси александр лукашенко заявил ...,президент беларусь александр лукашенко заявлят...
1,2,1,"['российские', 'компании', 'оказались', 'опасн...",российские компании оказались в опасности из з...,российский компания оказываться опасность глоб...,Несколько крупнейших российских компаний оказа...,несколько крупнейших российских компаний оказа...,несколько крупный российский компания оказыват...
2,3,0,"['лукашенко', 'объявил', 'отмене', 'выборов', ...",лукашенко объявил об отмене выборов из за напа...,лукашенко объявлять отмена выборы нападение ин...,Президент Белоруссии Александр Лукашенко объяв...,президент белоруссии александр лукашенко объяв...,президент белоруссия александр лукашенко объяв...
3,4,1,"['«роснефть»', 'стала', 'лидером', 'объему', '...",«роснефть» стала лидером по объему биржевых пр...,роснефть становиться лидер объем биржевой прод...,С начала июля нефтяные компании заметно увелич...,начала июля нефтяные компании заметно увеличил...,начинать июль нефтяной компания заметно увелич...
4,5,1,"['псковской', 'области', 'дадут', 'десятки', '...",псковской области дадут десятки миллионов рубл...,псковский область давать десяток миллион рубль...,Псковская область получит 189 миллионов рублей...,псковская область получит 189 миллионов рублей...,псковский область получать миллион рубль ремон...
...,...,...,...,...,...,...,...,...
1096,1097,0,"['борис', 'джонсон', 'пообещал', 'избирателям'...",борис джонсон пообещал избирателям вернуть ирл...,борис джонсон пообещать избиратель вернуть ирл...,Кандидат в премьер-министры Великобритании Бор...,кандидат премьер министры великобритании борис...,кандидат премьер министр великобритания борис ...
1097,1098,0,"['михаил', 'горбачев', 'заявил', 'подавал', 'о...",михаил горбачев заявил что не подавал в отстав...,михаил горбачев заявлять подавать отставка пос...,Сенсационное заявление сделал 8 декабря в эфир...,сенсационное заявление сделал 8 декабря эфире ...,сенсационный заявление сделать декабрь эфир не...
1098,1099,0,"['мгимо', 'появится', 'факультет', 'международ...",в мгимо появится факультет международных отнош...,мгимо появляться факультет международный отнош...,Московский государственный институт международ...,московский государственный институт международ...,московский государственный институт международ...
1099,1100,0,"['россии', 'введут', 'штраф', 'отрицание', 'аг...",в россии введут штраф за отрицание агрессии нато,россия вводить штраф отрицание агрессия нато,На рассмотрение в Государственную думу поступи...,рассмотрение государственную думу поступил зак...,рассмотрение государственный дума поступать за...


The training/test sample data are sparsed into a single file and are ready for further work with them.

# 1.2 EDA (exploratory data analysis)

In [7]:
data.info() # Let's get the information

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4408 entries, 0 to 4407
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Body ID       4408 non-null   int64 
 1   Headline      4408 non-null   object
 2   Stance        4408 non-null   object
 3   Headline1     4408 non-null   object
 4   Headline2     4408 non-null   object
 5   articleBody   4408 non-null   object
 6   articleBody1  4407 non-null   object
 7   articleBody2  4407 non-null   object
dtypes: int64(1), object(7)
memory usage: 275.6+ KB


In [8]:
data_test.info() # Let's get the information

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1101 entries, 0 to 1100
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Body ID       1101 non-null   int64 
 1   label         1101 non-null   int64 
 2   Headline      1101 non-null   object
 3   Headline1     1101 non-null   object
 4   Headline2     1101 non-null   object
 5   articleBody   1101 non-null   object
 6   articleBody1  1101 non-null   object
 7   articleBody2  1101 non-null   object
dtypes: int64(2), object(6)
memory usage: 68.9+ KB


Information about the data in the training/test sample:<li>4408/1101 entries</li>
<li>7/7 columns</li>
<li>1/2 signs of type 'int64'</li>
<li>7/6 signs of type 'object'</li>
<li>0/0 lost data</li></li>

Trait labeling:

<b>Unnamed: 0</b> - Line number

<b>Body ID</b> - news id

<b>Stance</b> - newsworthiness

<b>Headline</b> - original news title

<b>Headline1</b> - news title after the purge

<b>Headline2</b> - name of news after processing by PyMystem3 module

<b>articleBody</b> - original news content

<b>articleBody1</b> - news content after cleanup

<b>articleBody2</b> - news content after processing by PyMystem3 module

# 1.3 Data processing and word dictionary paging

In [9]:
# Codify the categorical variable
data['Stance'] = data['Stance'].replace(['agree', 'disagree'], [1, 0]).astype('int')

In [10]:
data # Let's take a look at the dataframe

,Body ID,Headline,Stance,Headline1,Headline2,articleBody,articleBody1,articleBody2
0,1,"['российский', 'бюджет', 'марте', 'недополучил...",1,российский бюджет в марте недополучил более 30...,российский бюджет март недополучать миллиард р...,Разница между ожидаемыми по итогам марта нефте...,разница ожидаемыми итогам марта нефтегазовыми ...,разница ожидать итог март нефтегазовый доход ф...
1,2,"['банк', 'россии', 'решил', 'снизить', 'ключев...",1,банк россии решил снизить ключевую ставку с 20...,банк россия решать снижать ключевой ставка,Совет директоров Банка России решил снизить кл...,совет директоров банка россии решил снизить кл...,совет директор банк россия решать снижать ключ...
2,3,"['мыс', 'идокопас', 'нато', 'назвали', 'первую...",0,мыс идокопас в нато назвали первую цель для н...,мыс идокопас нато называть первый цель начало ...,В НАТО составили наступательный план действий ...,нато составили наступательный план действий за...,нато составлять наступательный план действие з...
3,4,"['украине', 'прошли', 'празднования', '300', '...",0,на украине прошли празднования 300 летия побед...,украина проходить празднование летие победа ка...,Президент Владимир Зеленский принял в Полтаве ...,президент владимир зеленский принял полтаве па...,президент владимир зеленский принимать полтава...
4,5,"['минобороны', 'рф', 'заявило', 'ударах', 'рак...",1,минобороны рф заявило об ударах ракетами «кали...,минобороны заявлять удар ракета калибр запорож...,"Минобороны России объявило, что ракетами больш...",минобороны россии объявило ракетами большой да...,минобороны россия объявлять ракета большой дал...
...,...,...,...,...,...,...,...,...
4403,4404,"['чиновница', 'призвавшая', '«найти', 'покарат...",0,чиновница призвавшая «найти и покарать предате...,чиновница призывать находить покарать предател...,Заместитель министра просвещения Ирина Карепин...,заместитель министра просвещения ирина карепин...,заместитель министр просвещение ирина карепин ...
4404,4405,"['«включим', 'программу', 'киселёва', 'покажем...",0,«включим программу киселёва и покажем телевизо...,включать программа киселев показывать телевизо...,Глава штабов Навального Леонид Волков призвал ...,глава штабов навального леонид волков призвал ...,глава штаб навальный леонид волк призывать сво...
4405,4406,"['китайские', 'власти', 'могут', 'казнить', 'з...",0,китайские власти могут казнить задержанного по...,китайский власть мочь казнить задержанный подо...,Один из менеджеров стройки космодрома Восточны...,менеджеров стройки космодрома восточный взят с...,менеджер стройка космодром восточный взять стр...
4406,4407,"['луганской', 'области', 'переполнены', 'морги...",1,в луганской области переполнены морги хранить...,луганский область переполнять морг хранить тел...,Морги Луганской области переполнены телами пог...,морги луганской области переполнены телами пог...,морг луганский область переполнять тело погибш...


In [11]:
# Let's make the "Body ID" column an index
data = data.set_index("Body ID")
data

,Headline,Stance,Headline1,Headline2,articleBody,articleBody1,articleBody2
Body ID,,,,,,,
1,"['российский', 'бюджет', 'марте', 'недополучил...",1,российский бюджет в марте недополучил более 30...,российский бюджет март недополучать миллиард р...,Разница между ожидаемыми по итогам марта нефте...,разница ожидаемыми итогам марта нефтегазовыми ...,разница ожидать итог март нефтегазовый доход ф...
2,"['банк', 'россии', 'решил', 'снизить', 'ключев...",1,банк россии решил снизить ключевую ставку с 20...,банк россия решать снижать ключевой ставка,Совет директоров Банка России решил снизить кл...,совет директоров банка россии решил снизить кл...,совет директор банк россия решать снижать ключ...
3,"['мыс', 'идокопас', 'нато', 'назвали', 'первую...",0,мыс идокопас в нато назвали первую цель для н...,мыс идокопас нато называть первый цель начало ...,В НАТО составили наступательный план действий ...,нато составили наступательный план действий за...,нато составлять наступательный план действие з...
4,"['украине', 'прошли', 'празднования', '300', '...",0,на украине прошли празднования 300 летия побед...,украина проходить празднование летие победа ка...,Президент Владимир Зеленский принял в Полтаве ...,президент владимир зеленский принял полтаве па...,президент владимир зеленский принимать полтава...
5,"['минобороны', 'рф', 'заявило', 'ударах', 'рак...",1,минобороны рф заявило об ударах ракетами «кали...,минобороны заявлять удар ракета калибр запорож...,"Минобороны России объявило, что ракетами больш...",минобороны россии объявило ракетами большой да...,минобороны россия объявлять ракета большой дал...
...,...,...,...,...,...,...,...
4404,"['чиновница', 'призвавшая', '«найти', 'покарат...",0,чиновница призвавшая «найти и покарать предате...,чиновница призывать находить покарать предател...,Заместитель министра просвещения Ирина Карепин...,заместитель министра просвещения ирина карепин...,заместитель министр просвещение ирина карепин ...
4405,"['«включим', 'программу', 'киселёва', 'покажем...",0,«включим программу киселёва и покажем телевизо...,включать программа киселев показывать телевизо...,Глава штабов Навального Леонид Волков призвал ...,глава штабов навального леонид волков призвал ...,глава штаб навальный леонид волк призывать сво...
4406,"['китайские', 'власти', 'могут', 'казнить', 'з...",0,китайские власти могут казнить задержанного по...,китайский власть мочь казнить задержанный подо...,Один из менеджеров стройки космодрома Восточны...,менеджеров стройки космодрома восточный взят с...,менеджер стройка космодром восточный взять стр...


In [12]:
# Let's parse the "Headline" column
data['Headline'] = data['Headline'].apply(lambda x: "".join(x.replace("[", "").replace("]", "").replace("'", "").split(',')))
data

,Headline,Stance,Headline1,Headline2,articleBody,articleBody1,articleBody2
Body ID,,,,,,,
1,российский бюджет марте недополучил 300 миллиа...,1,российский бюджет в марте недополучил более 30...,российский бюджет март недополучать миллиард р...,Разница между ожидаемыми по итогам марта нефте...,разница ожидаемыми итогам марта нефтегазовыми ...,разница ожидать итог март нефтегазовый доход ф...
2,банк россии решил снизить ключевую ставку 20 17,1,банк россии решил снизить ключевую ставку с 20...,банк россия решать снижать ключевой ставка,Совет директоров Банка России решил снизить кл...,совет директоров банка россии решил снизить кл...,совет директор банк россия решать снижать ключ...
3,мыс идокопас нато назвали первую цель начала п...,0,мыс идокопас в нато назвали первую цель для н...,мыс идокопас нато называть первый цель начало ...,В НАТО составили наступательный план действий ...,нато составили наступательный план действий за...,нато составлять наступательный план действие з...
4,украине прошли празднования 300 летия победы к...,0,на украине прошли празднования 300 летия побед...,украина проходить празднование летие победа ка...,Президент Владимир Зеленский принял в Полтаве ...,президент владимир зеленский принял полтаве па...,президент владимир зеленский принимать полтава...
5,минобороны рф заявило ударах ракетами «калибр»...,1,минобороны рф заявило об ударах ракетами «кали...,минобороны заявлять удар ракета калибр запорож...,"Минобороны России объявило, что ракетами больш...",минобороны россии объявило ракетами большой да...,минобороны россия объявлять ракета большой дал...
...,...,...,...,...,...,...,...
4404,чиновница призвавшая «найти покарать предателе...,0,чиновница призвавшая «найти и покарать предате...,чиновница призывать находить покарать предател...,Заместитель министра просвещения Ирина Карепин...,заместитель министра просвещения ирина карепин...,заместитель министр просвещение ирина карепин ...
4405,«включим программу киселёва покажем телевизора...,0,«включим программу киселёва и покажем телевизо...,включать программа киселев показывать телевизо...,Глава штабов Навального Леонид Волков призвал ...,глава штабов навального леонид волков призвал ...,глава штаб навальный леонид волк призывать сво...
4406,китайские власти могут казнить задержанного по...,0,китайские власти могут казнить задержанного по...,китайский власть мочь казнить задержанный подо...,Один из менеджеров стройки космодрома Восточны...,менеджеров стройки космодрома восточный взят с...,менеджер стройка космодром восточный взять стр...


In [13]:
data_test['Stance'] = data_test['label'] # Rename the target metric
data_test.drop('label', axis=1, inplace=True)
data_test

,Body ID,Headline,Headline1,Headline2,articleBody,articleBody1,articleBody2,Stance
0,1,"['лукашенко', 'пригрозил', 'литовским', 'танка...",лукашенко пригрозил литовским танкам белорусск...,лукашенко пригрозить литовский танк белорусски...,"Президент Беларуси Александр Лукашенко заявил,...",президент беларуси александр лукашенко заявил ...,президент беларусь александр лукашенко заявлят...,0
1,2,"['российские', 'компании', 'оказались', 'опасн...",российские компании оказались в опасности из з...,российский компания оказываться опасность глоб...,Несколько крупнейших российских компаний оказа...,несколько крупнейших российских компаний оказа...,несколько крупный российский компания оказыват...,1
2,3,"['лукашенко', 'объявил', 'отмене', 'выборов', ...",лукашенко объявил об отмене выборов из за напа...,лукашенко объявлять отмена выборы нападение ин...,Президент Белоруссии Александр Лукашенко объяв...,президент белоруссии александр лукашенко объяв...,президент белоруссия александр лукашенко объяв...,0
3,4,"['«роснефть»', 'стала', 'лидером', 'объему', '...",«роснефть» стала лидером по объему биржевых пр...,роснефть становиться лидер объем биржевой прод...,С начала июля нефтяные компании заметно увелич...,начала июля нефтяные компании заметно увеличил...,начинать июль нефтяной компания заметно увелич...,1
4,5,"['псковской', 'области', 'дадут', 'десятки', '...",псковской области дадут десятки миллионов рубл...,псковский область давать десяток миллион рубль...,Псковская область получит 189 миллионов рублей...,псковская область получит 189 миллионов рублей...,псковский область получать миллион рубль ремон...,1
...,...,...,...,...,...,...,...,...
1096,1097,"['борис', 'джонсон', 'пообещал', 'избирателям'...",борис джонсон пообещал избирателям вернуть ирл...,борис джонсон пообещать избиратель вернуть ирл...,Кандидат в премьер-министры Великобритании Бор...,кандидат премьер министры великобритании борис...,кандидат премьер министр великобритания борис ...,0
1097,1098,"['михаил', 'горбачев', 'заявил', 'подавал', 'о...",михаил горбачев заявил что не подавал в отстав...,михаил горбачев заявлять подавать отставка пос...,Сенсационное заявление сделал 8 декабря в эфир...,сенсационное заявление сделал 8 декабря эфире ...,сенсационный заявление сделать декабрь эфир не...,0
1098,1099,"['мгимо', 'появится', 'факультет', 'международ...",в мгимо появится факультет международных отнош...,мгимо появляться факультет международный отнош...,Московский государственный институт международ...,московский государственный институт международ...,московский государственный институт международ...,0
1099,1100,"['россии', 'введут', 'штраф', 'отрицание', 'аг...",в россии введут штраф за отрицание агрессии нато,россия вводить штраф отрицание агрессия нато,На рассмотрение в Государственную думу поступи...,рассмотрение государственную думу поступил зак...,рассмотрение государственный дума поступать за...,0


In [14]:
# Let's make the "Body ID" column an index
data_test = data_test.set_index("Body ID")
data_test

,Headline,Headline1,Headline2,articleBody,articleBody1,articleBody2,Stance
Body ID,,,,,,,
1,"['лукашенко', 'пригрозил', 'литовским', 'танка...",лукашенко пригрозил литовским танкам белорусск...,лукашенко пригрозить литовский танк белорусски...,"Президент Беларуси Александр Лукашенко заявил,...",президент беларуси александр лукашенко заявил ...,президент беларусь александр лукашенко заявлят...,0
2,"['российские', 'компании', 'оказались', 'опасн...",российские компании оказались в опасности из з...,российский компания оказываться опасность глоб...,Несколько крупнейших российских компаний оказа...,несколько крупнейших российских компаний оказа...,несколько крупный российский компания оказыват...,1
3,"['лукашенко', 'объявил', 'отмене', 'выборов', ...",лукашенко объявил об отмене выборов из за напа...,лукашенко объявлять отмена выборы нападение ин...,Президент Белоруссии Александр Лукашенко объяв...,президент белоруссии александр лукашенко объяв...,президент белоруссия александр лукашенко объяв...,0
4,"['«роснефть»', 'стала', 'лидером', 'объему', '...",«роснефть» стала лидером по объему биржевых пр...,роснефть становиться лидер объем биржевой прод...,С начала июля нефтяные компании заметно увелич...,начала июля нефтяные компании заметно увеличил...,начинать июль нефтяной компания заметно увелич...,1
5,"['псковской', 'области', 'дадут', 'десятки', '...",псковской области дадут десятки миллионов рубл...,псковский область давать десяток миллион рубль...,Псковская область получит 189 миллионов рублей...,псковская область получит 189 миллионов рублей...,псковский область получать миллион рубль ремон...,1
...,...,...,...,...,...,...,...
1097,"['борис', 'джонсон', 'пообещал', 'избирателям'...",борис джонсон пообещал избирателям вернуть ирл...,борис джонсон пообещать избиратель вернуть ирл...,Кандидат в премьер-министры Великобритании Бор...,кандидат премьер министры великобритании борис...,кандидат премьер министр великобритания борис ...,0
1098,"['михаил', 'горбачев', 'заявил', 'подавал', 'о...",михаил горбачев заявил что не подавал в отстав...,михаил горбачев заявлять подавать отставка пос...,Сенсационное заявление сделал 8 декабря в эфир...,сенсационное заявление сделал 8 декабря эфире ...,сенсационный заявление сделать декабрь эфир не...,0
1099,"['мгимо', 'появится', 'факультет', 'международ...",в мгимо появится факультет международных отнош...,мгимо появляться факультет международный отнош...,Московский государственный институт международ...,московский государственный институт международ...,московский государственный институт международ...,0


In [15]:
# Let's parse the "Headline" column
data_test['Headline'] = data_test['Headline'].apply(lambda x: "".join(x.replace("[", "").replace("]", "").replace("'", "").split(',')))
data_test

,Headline,Headline1,Headline2,articleBody,articleBody1,articleBody2,Stance
Body ID,,,,,,,
1,лукашенко пригрозил литовским танкам белорусск...,лукашенко пригрозил литовским танкам белорусск...,лукашенко пригрозить литовский танк белорусски...,"Президент Беларуси Александр Лукашенко заявил,...",президент беларуси александр лукашенко заявил ...,президент беларусь александр лукашенко заявлят...,0
2,российские компании оказались опасности глобал...,российские компании оказались в опасности из з...,российский компания оказываться опасность глоб...,Несколько крупнейших российских компаний оказа...,несколько крупнейших российских компаний оказа...,несколько крупный российский компания оказыват...,1
3,лукашенко объявил отмене выборов нападения ино...,лукашенко объявил об отмене выборов из за напа...,лукашенко объявлять отмена выборы нападение ин...,Президент Белоруссии Александр Лукашенко объяв...,президент белоруссии александр лукашенко объяв...,президент белоруссия александр лукашенко объяв...,0
4,«роснефть» стала лидером объему биржевых прода...,«роснефть» стала лидером по объему биржевых пр...,роснефть становиться лидер объем биржевой прод...,С начала июля нефтяные компании заметно увелич...,начала июля нефтяные компании заметно увеличил...,начинать июль нефтяной компания заметно увелич...,1
5,псковской области дадут десятки миллионов рубл...,псковской области дадут десятки миллионов рубл...,псковский область давать десяток миллион рубль...,Псковская область получит 189 миллионов рублей...,псковская область получит 189 миллионов рублей...,псковский область получать миллион рубль ремон...,1
...,...,...,...,...,...,...,...
1097,борис джонсон пообещал избирателям вернуть ирл...,борис джонсон пообещал избирателям вернуть ирл...,борис джонсон пообещать избиратель вернуть ирл...,Кандидат в премьер-министры Великобритании Бор...,кандидат премьер министры великобритании борис...,кандидат премьер министр великобритания борис ...,0
1098,михаил горбачев заявил подавал отставку поста ...,михаил горбачев заявил что не подавал в отстав...,михаил горбачев заявлять подавать отставка пос...,Сенсационное заявление сделал 8 декабря в эфир...,сенсационное заявление сделал 8 декабря эфире ...,сенсационный заявление сделать декабрь эфир не...,0
1099,мгимо появится факультет международных отношен...,в мгимо появится факультет международных отнош...,мгимо появляться факультет международный отнош...,Московский государственный институт международ...,московский государственный институт международ...,московский государственный институт международ...,0


In [16]:
# Pumping up the vocabulary of words
nltk.download("stopwords")
russian_stopwords = stopwords.words("russian")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\k1rsn\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


The data is ready to be transferred to the models

# 2.1 Training out-of-the-box models

In [17]:
def fit_predict_score_model(df, model, df_test):
    """
    Input:
        df - training dataframe with headline and news text
        model - machine learning model
        df_test - test dataframe with headline and news text
    Return:
        The first parameter returns the accuracy of model classification on news headlines
        The second parameter returns the accuracy of model classification on news content
    """
    vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=russian_stopwords)
    columns = df.drop('Stance', axis=1).columns # Получаем список признаков
    print("-"*30)
    print(f"   A couple of signs: {columns[0]} и {columns[1]}")
    
    # Teach CountVectorizer on all news stories
    vectorizer.fit(df[columns[0]].values.astype('U'))
    vectorizer.fit(df[columns[1]].values.astype('U'))
    vectorizer.fit(df_test[columns[0]].values.astype('U'))
    vectorizer.fit(df_test[columns[1]].values.astype('U'))

    # Transform the text into an array of word id's
    X11 = vectorizer.transform(df[columns[0]].values.astype('U')).toarray()
    X12 = vectorizer.transform(df[columns[1]].values.astype('U')).toarray()
    X21 = vectorizer.transform(df_test[columns[0]].values.astype('U')).toarray()
    X22 = vectorizer.transform(df_test[columns[1]].values.astype('U')).toarray()

    # Train the model on news headlines and look at accuracy
    print('   Learning from the headlines')
    model_new = model
    model_new.fit(X11, df.Stance)
    preds1 = model_new.predict(X21)
    print(f"      Accuracy on the headlines: {accuracy_score(df_test.Stance, preds1)}")

    #  Train the model on news content and look at accuracy
    print('   Training for content')
    model_new = model
    model_new.fit(X12, df.Stance)
    preds2 = model_new.predict(X22)
    print(f"      Accuracy on content: {accuracy_score(df_test.Stance, preds2)}")
    
    return accuracy_score(df_test.Stance, preds1), accuracy_score(df_test.Stance, preds2)

In [18]:
# Form a DataFrame for model accuracy records
accuracy_df = pd.DataFrame(columns=['Name Model', 'Headline', 'articleBody', 'Headline1', 'articleBody1', 'Headline2', 'articleBody2'])

i = 0
for model in [LogisticRegression(), RandomForestClassifier(), DecisionTreeClassifier(), SVC()]: 
    # Vocabulary for filling in the DataFrame with model accuracies
    accuracy_list = {'Name Model':None, 'Headline':None, 'articleBody':None, 'Headline1':None, 'articleBody1':None, 
                     'Headline2':None, 'articleBody2':None}
    print("*"*30)
    print(model.__class__.__name__)
    print("*"*30)

    # We train the model on the original data and write to the dictionary
    itog1, itog2 = fit_predict_score_model(data[['Headline', 'articleBody', 'Stance']], model, data_test[['Headline', 'articleBody', 'Stance']])
    accuracy_list['Name Model'] = model.__class__.__name__
    accuracy_list['Headline'] = itog1
    accuracy_list['articleBody'] = itog2

    # We train the model on the post-cleaning data and write to the dictionary
    itog1, itog2 = fit_predict_score_model(data[['Headline1', 'articleBody1', 'Stance']], model, data_test[['Headline1', 'articleBody1', 'Stance']])
    accuracy_list['Headline1'] = itog1
    accuracy_list['articleBody1'] = itog2
    
    # We train the model on the data after processing by PyMystem3 module and write it to the dictionary
    itog1, itog2 = fit_predict_score_model(data[['Headline2', 'articleBody2', 'Stance']], model, data_test[['Headline2', 'articleBody2', 'Stance']])
    accuracy_list['Headline2'] = itog1
    accuracy_list['articleBody2'] = itog2

    # Writing model data to the dataframe
    accuracy_df = pd.concat([accuracy_df, pd.DataFrame(accuracy_list, index=[i])], ignore_index=True)
    i += 1

******************************
LogisticRegression
******************************
------------------------------
   A couple of signs: Headline и articleBody
   Learning from the headlines
      Accuracy on the headlines: 0.740236148955495
   Training for content
      Accuracy on content: 0.9346049046321526
------------------------------
   A couple of signs: Headline1 и articleBody1
   Learning from the headlines
      Accuracy on the headlines: 0.737511353315168
   Training for content
      Accuracy on content: 0.9346049046321526
------------------------------
   A couple of signs: Headline2 и articleBody2
   Learning from the headlines
      Accuracy on the headlines: 0.7938237965485921
   Training for content
      Accuracy on content: 0.932788374205268
******************************
RandomForestClassifier
******************************
------------------------------
   A couple of signs: Headline и articleBody
   Learning from the headlines
      Accuracy on the headlines: 0.7420

Information on the accuracy of all "out of the box" models for each news data type is obtained

# 2.2 Identifying the model with the best accuracy

In [19]:
accuracy_df = accuracy_df.set_index('Name Model') # Changing indexes to model names

In [20]:
accuracy_df # Looking at the data

,Headline,articleBody,Headline1,articleBody1,Headline2,articleBody2
Name Model,,,,,,
LogisticRegression,0.740236,0.934605,0.737511,0.934605,0.793824,0.932788
RandomForestClassifier,0.742053,0.925522,0.742961,0.920073,0.784741,0.920073
DecisionTreeClassifier,0.689373,0.833787,0.693006,0.831063,0.734787,0.815622
SVC,0.752044,0.944596,0.752044,0.944596,0.806540,0.937330


In [21]:
# We compare the average accuracy of the model for each of the attributes
round(accuracy_df.mean().sort_values(ascending=0), 3) 

articleBody     0.910
articleBody1    0.908
articleBody2    0.901
Headline2       0.780
Headline1       0.731
Headline        0.731
dtype: float64

It can be seen that the original news content gives the most informative content after cleaning, which helps to categorize credibility more accurately

In [22]:
# We compare the accuracy of the model by a more informative feature
round(accuracy_df['articleBody1'].sort_values(ascending=0)*100, 3)

Name Model
SVC                       94.460
LogisticRegression        93.460
RandomForestClassifier    92.007
DecisionTreeClassifier    83.106
Name: articleBody1, dtype: float64

The SVC model showed the best out-of-the-box accuracy. This is the model that will be best trained

# 2.3 Learning hyperparameters from the best model 

In [23]:
# Hyperparameters for training
param_grid = [
  {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
  {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
 ]

# Initialize the model with hyperparameters auto-selection
clf = GridSearchCV(estimator=SVC(), param_grid=param_grid, cv=3,  n_jobs=-1)

In [24]:
# Preprocessing the data for the model
vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=russian_stopwords)
vectorizer.fit(data['articleBody1'].values.astype('U'))         
vectorizer.fit(data_test['articleBody1'].values.astype('U'))   
X_train = vectorizer.transform(data['articleBody1'].values.astype('U')).toarray()         
X_test = vectorizer.transform(data_test['articleBody1'].values.astype('U')).toarray()    

In [25]:
%%time
clf.fit(X_train, data.Stance) # We teach all possible models 
svc_best = clf.best_estimator_ # We get the best copy
svc_best.fit(X_train, data.Stance) # Teaching the best exemplar
preds = svc_best.predict(X_test) # Predicting
print(f"Accuracy on test: {round(accuracy_score(data_test.Stance, preds), 3)}%")

Accuracy on test: 0.95%
CPU times: total: 16.5 s
Wall time: 1min 43s


As a result, we were able to raise the accuracy by 0.54% and achieve an accuracy of 95%, which is an excellent result